<a href="https://colab.research.google.com/github/balezz/cv_course_fa_mag/blob/main/CV_codelab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer vision
### CodeLab 1

Изображение в пространстве: основные 
статистики, пространственные и временные меры 
данных, ступенчато-граничная модель.

Цветовые модели. Изменение цветового 
пространства.


Геометрические преобразования изображения.


Методы выравнивания гистограммы.

 Фильтры Гаусса, Собеля, Прюит, медианный фильтр. Способы 
создания/устранения шума.